In [13]:
import os
import json
import pandas as pd
import seaborn as sns
import web3
from web3 import Web3
from dotenv import load_dotenv

load_dotenv() #TODO

False

In [8]:
ETHERSCAN_KEY = #TODO

SyntaxError: invalid syntax (965663091.py, line 1)

In [9]:
df = pd.DataFrame(columns=["block_timestamp", "address", "protocol", "deposit"])
df.head()

,block_timestamp,address,protocol,deposit


In [10]:
w3 = Web3(Web3.HTTPProvider("https://eth-mainnet.g.alchemy.com/v2/pKBWxMjs5nelNDeGydaC1J6XOWsn14_L"))

In [18]:
beacon_deposits_contract_address = "0x00000000219ab540356cBB839Cbe05303d7705Fa"
with open(os.path.join("abi", "beacon.json"), "r") as f:
    beacon_abi = json.load(f)
beacon_deposits = w3.eth.contract(beacon_deposits_contract_address, abi=beacon_abi)



In [22]:
from_block = 11052984 # beacon deployment block
to_block = 19040000

while from_block < to_block:
    logs = beacon_deposits.events.DepositEvent().get_logs(fromBlock=from_block, toBlock=from_block+2000)
    print(logs)

ValueError: {'code': -32602, 'message': 'Log response size exceeded. You can make eth_getLogs requests with up to a 2K block range and no limit on the response size, or you can request any block range with a cap of 10K logs in the response. Based on your parameters and the response size limit, this block range should work: [0xa8a7b8, 0xac6e91]'}

In [ ]:
# Evaluate if address is a contract